**Необходимые импорты**

In [1]:
import numpy as np
import cv2
import argparse
import math
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(2)
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from PIL import Image, ImageChops, ImageEnhance, ImageOps
import os
import itertools

**Инверсия изображения**

In [2]:
def invert(image):
    return ImageOps.invert(image)

**Изменение контрастности**

In [3]:
def change_contrast(img, level):
    factor = (259 * (level + 255)) / (255 * (259 - level))
    def contrast(c):
        return 128 + factor * (c - 128)
    return img.point(contrast)

**Гамма-коррекция**

In [4]:
#def gamma(image):
    #image = cv2.imread(image)
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #mid = 0.5
    #mean = np.mean(gray)
    #gamma = math.log(mid*255)/math.log(mean)
    #img_gamma = np.power(image, gamma).clip(0,255).astype(np.uint8)
    #return(img_gamma)

In [5]:
#def gammaCorrection(src, gamma):
    #invGamma = 1 / gamma
 
    #table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    #table = np.array(table, np.uint8)
 
    #return cv2.LUT(src, table)

In [6]:
 #def gammaCorrection(image):
        #gamma = np.random.uniform(low=0, high=3, size=[1,])
        #gain = np.random.uniform(low=0, high=1, size=[1,])
        #r = tf.image.adjust_gamma(image, gamma=gamma[0], gain=gain[0])
        #return r

In [7]:
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")

    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

**Анализ уровня ошибок**

In [8]:
def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpg'
    ela_filename = 'temp_ela.png'
    
    image = Image.open(path).convert('RGB')
    
    
    image = change_contrast(image, 100)
    brightness_factor = 1.5
    enhancer = ImageEnhance.Brightness(image)
    img = enhancer.enhance(brightness_factor)
    image = change_contrast(img, 100)
    image = invert(image)
    
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)
    
    ela_image = ImageChops.difference(image, temp_image)
    
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
#     brightness_factor = 1.5
#     enhancer = ImageEnhance.Brightness(ela_image)
#     img = enhancer.enhance(brightness_factor)
#     ela_image = change_contrast(img, 35)
    
    return ela_image

In [9]:
# def convert_to_ela_image(path, quality):
#     temp_filename = 'temp_file_name.jpg'
#     ela_filename = 'temp_ela.png'
    
#     image = Image.open(path).convert('RGB')
#     image.save(temp_filename, 'JPEG', quality = quality)
#     temp_image = Image.open(temp_filename)
    
#     ela_image = ImageChops.difference(image, temp_image)
    
#     extrema = ela_image.getextrema()
#     max_diff = max([ex[1] for ex in extrema])
#     if max_diff == 0:
#         max_diff = 1
#     scale = 255.0 / max_diff
    
#     ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
#     return ela_image

**Тест ELA**

In [10]:
real_image_path = '/kaggle/input/photoshop-detection-dataset/CASIA2 modified/Unedited/Au_ani_00001.jpg'
Image.open(real_image_path)

In [11]:
from PIL import Image, ImageFilter

image = Image.open(real_image_path)
edges_image = image.filter(ImageFilter.FIND_EDGES)
edges_image

In [12]:
convert_to_ela_image(real_image_path, 90)

In [13]:
fake_image_path = '/kaggle/input/photoshop-detection-dataset/CASIA2 modified/Edited/VhCle-LuD_Y.jpg'
Image.open(fake_image_path)

In [14]:
image_1 = convert_to_ela_image(fake_image_path, 90)
image_2 = convert_to_ela_image(fake_image_path, 95)
#edges_image = image.filter(ImageFilter.FIND_EDGES)
#brightness_factor = 1.5
#enhancer = ImageEnhance.Brightness(image)
#img = enhancer.enhance(brightness_factor)
#image = change_contrast(img, 35)
#image_1
image_2
#edges_image

In [15]:
image = Image.open(fake_image_path)
image = change_contrast(image, 100)
edges_image = image.filter(ImageFilter.FIND_EDGES)
edges_image

In [16]:
edge_detector = np.array([[-1,-1,-1],
                          [-1, 8,-1],
                          [-1,-1,-1]])

fake_image = image.filter(ImageFilter.Kernel(size=(3,3), kernel=edge_detector.flatten(), scale=0.2))

display(fake_image)

**Подготовка датасета**

In [17]:
image_size = (128, 128)

In [18]:
def prepare_image(image_path):
    return np.array(convert_to_ela_image(image_path, 95).resize(image_size)).flatten() / 255.0

In [19]:
X = [] # Изображения после ELA
Y = [] # 0 - отфотошопленные, 1 - оригинальные

**Выбор случайных изображений**

In [20]:
import random
path = '/kaggle/input/photoshop-detection-dataset/CASIA2 modified/Unedited'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('jpg') or filename.endswith('png') or filename.endswith('tif'):
            full_path = os.path.join(dirname, filename)
            X.append(prepare_image(full_path))
            Y.append(1)
            if len(Y) % 200 == 0:
                print(f'Processing {len(Y)} images')

random.shuffle(X)
X = X[:1000]
Y = Y[:1000]
print(len(X), len(Y))

In [21]:
path = '/kaggle/input/photoshop-detection-dataset/CASIA2 modified/Edited'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('jpg') or filename.endswith('png') or filename.endswith('tif'):
            full_path = os.path.join(dirname, filename)
            X.append(prepare_image(full_path))
            Y.append(0)
            if len(Y) % 200 == 0:
                print(f'Processing {len(Y)} images')

print(len(X), len(Y))

In [22]:
X = np.array(X)
Y = to_categorical(Y, 2)
X = X.reshape(-1, 128, 128, 3)

**Train/Test - 80/20**

In [23]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)
X = X.reshape(-1,1,1,1)
print(len(X_train), len(Y_train))
print(len(X_val), len(Y_val))
print(X)

**Построение модели свёрточной нейронной сети**

In [24]:
def build_model():
    model = Sequential()
    model.add(Conv2D(input_shape=(128,128,3),filters=64,kernel_size=(2,2),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(2,2),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(2,2), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(2,2), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(2,2), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(2,2), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(2,2), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(2,2), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(2,2), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(2,2), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(2,2), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(2,2), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(2,2), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=2, activation="softmax"))
    return model

In [25]:
model = build_model()
model.summary()

In [26]:
epochs = 10
batch_size = 32

In [27]:
init_lr = 1e-4
optimizer = Adam(lr = init_lr, decay = init_lr/epochs)

In [28]:
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [29]:
early_stopping = EarlyStopping(monitor = 'val_acc',
                              min_delta = 0,
                              patience = 2,
                              verbose = 0,
                              mode = 'auto')

In [30]:
hist = model.fit(X_train,
                 Y_train,
                 batch_size = batch_size,
                 epochs = epochs,
                validation_data = (X_val, Y_val),
                callbacks = [early_stopping])

In [31]:
model.save('model_Test Photoshop detection VGG-16 CASIA2_modified lre-4 2000 GPU.h5')

**Метрики**

In [32]:
fig, ax = plt.subplots(2,1)
fig.set_size_inches(10, 5)
fig.set_dpi(100)
ax[0].plot(hist.history['loss'], color='b', label="Training loss")
ax[0].plot(hist.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(hist.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(hist.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [33]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [34]:
# Predict the values from the validation dataset
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(2))

In [35]:
print(Y_pred)

In [36]:
print(Y_pred_classes)

In [37]:
print(Y_true)

In [38]:
print(Y_val)

In [39]:
#print(X_val)

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_true, Y_pred_classes)

In [41]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(Y_true, Y_pred_classes)

In [42]:
from sklearn.metrics import top_k_accuracy_score
top_k_accuracy_score(Y_true, Y_pred_classes, k=2)

In [43]:
from sklearn.metrics import average_precision_score
average_precision_score(Y_true, Y_pred_classes)

In [44]:
from sklearn.metrics import recall_score
recall_score(Y_true, Y_pred_classes, average='macro')

In [45]:
from sklearn.metrics import f1_score
f1_score(Y_true, Y_pred_classes, average='macro')

In [46]:
f1_score(Y_true, Y_pred_classes, average='micro')

In [47]:
f1_score(Y_true, Y_pred_classes, average='weighted')

In [48]:
def pretty_table(data, cell_sep=' | ', header_separator=True) -> str:
    rows = len(data)
    cols = len(data[0])

    col_width = []
    for col in range(cols):
        columns = [str(data[row][col]) for row in range(rows)]
        col_width.append(len(max(columns, key=len)))

    separator = "-+-".join('-' * n for n in col_width)

    lines = []

    for i, row in enumerate(range(rows)):
        result = []
        for col in range(cols):
            item = str(data[row][col]).rjust(col_width[col])
            result.append(item)

        lines.append(cell_sep.join(result))

        if i == 0 and header_separator:
            lines.append(separator)

    return '\n'.join(lines)

In [49]:
data = [
    ["Метрика", "Значение"],
    {"Accuracy": round(accuracy_score(Y_true, Y_pred_classes), 3), 
     "Balanced accuracy": round(balanced_accuracy_score(Y_true, Y_pred_classes), 3), 
     "Average precision score": round(average_precision_score(Y_true, Y_pred_classes), 3),
     "Recall": round(recall_score(Y_true, Y_pred_classes, average='macro'), 3),
     "F1": round(f1_score(Y_true, Y_pred_classes, average='macro'), 3),}
]
rows = [
    data[0]
]
rows += [(k, v) for k, v in data[1].items()]
print(pretty_table(rows))

Accuracy

In [50]:
from matplotlib import pyplot

pyplot.plot(hist.history['accuracy'])
pyplot.title('Accuracy')
pyplot.show()

ROC-AUC

In [51]:
predictions = []
for i in Y_pred:
    predictions.append(np.amax(i)*100)

In [52]:
#print(predictions)
#print(Y_true)

In [53]:
from sklearn.metrics import roc_curve  
 
y = Y_true
pred = predictions
fpr, tpr, thresholds = roc_curve(y, pred, pos_label=1) 
#print('fpr: ' + str(fpr))
#print('tpr: ' + str(tpr))
#print('thresholds: ' + str(thresholds))

from sklearn.metrics import auc  
AUC = auc(fpr, tpr) 
#print('AUC: ' + str(AUC))

plt.plot(fpr, tpr)  
plt.title('ROC_curve' + '(AUC: ' + str(AUC) + ')' )
plt.ylabel('True Positive Rate')  
plt.xlabel('False Positive Rate')
pyplot.plot([0, 1], [0, 1], linestyle='--')
plt.show()  

Precision-Recall

In [54]:
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(Y_true, predictions)

#create precision recall curve
fig, ax = plt.subplots()
ax.plot(recall, precision, color='purple')

#add axis labels to plot
ax.set_title('Precision-Recall Curve')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')

#display plot
plt.show()

**Предсказание**

In [55]:
class_names = ['fake', 'real']

In [56]:
real_image_path = '/kaggle/input/photoshop-detection-dataset/CASIA2 modified/Unedited/Au_ani_00001.jpg'
image = prepare_image(real_image_path)
image = image.reshape(-1, 128, 128, 3)
y_pred = model.predict(image)
y_pred_class = np.argmax(y_pred, axis = 1)[0]
print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [57]:
fake_image_path = '/kaggle/input/photoshop-detection-dataset/CASIA2 modified/Edited/beach_wood_copy.png'
image = prepare_image(fake_image_path)
image = image.reshape(-1, 128, 128, 3)
y_pred = model.predict(image)
y_pred_class = np.argmax(y_pred, axis = 1)[0]
print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [ ]:
fake_image = os.listdir('/kaggle/input/photoshop-detection-dataset/CASIA2 modified/Edited/')
correct = 0
total = 0
for file_name in fake_image:
    if file_name.endswith('jpg') or filename.endswith('png') or filename.endswith('tif'):
        fake_image_path = os.path.join('/kaggle/input/photoshop-detection-dataset/CASIA2 modified/Edited/', file_name)
        image = prepare_image(fake_image_path)
        image = image.reshape(-1, 128, 128, 3)
        y_pred = model.predict(image)
        y_pred_class = np.argmax(y_pred, axis = 1)[0]
        total += 1
        if y_pred_class == 0:
            correct += 1
#             print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [ ]:
print(f'Total: {total}, Correct: {correct}, Acc: {correct / total * 100.0}')

In [ ]:
real_image = os.listdir('/kaggle/input/photoshop-detection-dataset/CASIA2 modified/Unedited/')
correct_r = 0
total_r = 0
for file_name in real_image:
    if file_name.endswith('jpg') or filename.endswith('png') or filename.endswith('tif'):
        real_image_path = os.path.join('/kaggle/input/photoshop-detection-dataset/CASIA2 modified/Unedited/', file_name)
        image = prepare_image(real_image_path)
        image = image.reshape(-1, 128, 128, 3)
        y_pred = model.predict(image)
        y_pred_class = np.argmax(y_pred, axis = 1)[0]
        total_r += 1
        if y_pred_class == 1:
            correct_r += 1
#             print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [ ]:
correct += correct_r
total += total_r
print(f'Total: {total_r}, Correct: {correct_r}, Acc: {correct_r / total_r * 100.0}')
print(f'Total: {total}, Correct: {correct}, Acc: {correct / total * 100.0}')